In [1]:
%reset -f
import sys
sys.path.insert(0,'..')
from wavhandler import *
# from utils_train import train_test_val_split
from pandas.plotting import register_matplotlib_converters
from utils_train import test_inds, test_days
register_matplotlib_converters()
import numpy as np

np.random.seed(42)

In [2]:
data = Dataset('Wingbeats')
print(data.target_classes)

['Ae. aegypti', 'Ae. albopictus', 'An. arabiensis', 'An. gambiae', 'C. pipiens', 'C. quinquefasciatus']


In [3]:
data.read('Ae. aegypti', loadmat=False)
x1 = data.filenames.sample(14800)
data.read('Ae. albopictus', loadmat=False)
x2 = data.filenames.sample(14800)
data.read('An. arabiensis', loadmat=False)
x3 = data.filenames.sample(14800)
data.read('An. gambiae', loadmat=False)
x4 = data.filenames.sample(14800)
data.read('C. pipiens', loadmat=False)
x5 = data.filenames.sample(14800)
data.read('C. quinquefasciatus', loadmat=False)
x6 = data.filenames.sample(14800)

Data: Ae. aegypti.
Read 85553 filenames in 2.36 seconds.
Data: Ae. albopictus.
Read 20231 filenames in 2.30 seconds.
Data: An. arabiensis.
Read 19297 filenames in 2.27 seconds.
Data: An. gambiae.
Read 49471 filenames in 2.26 seconds.
Data: C. pipiens.
Read 30415 filenames in 2.25 seconds.
Data: C. quinquefasciatus.
Read 74599 filenames in 2.25 seconds.


In [18]:
X = pd.concat([x1, x2, x3, x4, x5, x6], axis=0)
y = X.apply(lambda x: x.split('/')[len(BASE_DIR.split('/'))])

In [19]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

In [20]:
from sklearn.utils import shuffle
from utils_train import train_test_val_split

X,y = shuffle(X.tolist(),y.tolist(), random_state=0)

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X,y,test_size=0.13514, val_size=0.2)

In [21]:
keys = pd.Series(le.inverse_transform(y_train)).value_counts().index.tolist()
values = pd.Series(y_train).value_counts().index.tolist()
mapping = dict(zip(keys, values))
print(sorted(mapping.items(), key=lambda x: x[1]))
vcounts = pd.Series(y_train).value_counts()
vcounts.index = mapping.keys()
vcounts

[('An. gambiae', 0), ('Ae. albopictus', 1), ('Ae. aegypti', 2), ('An. arabiensis', 3), ('C. pipiens', 4), ('C. quinquefasciatus', 5)]


C. quinquefasciatus    10240
An. arabiensis         10240
Ae. aegypti            10240
Ae. albopictus         10240
An. gambiae            10240
C. pipiens             10239
dtype: int64

In [22]:
df_train = make_df_parallel(setting='raw', names=X_train+X_val)
# df_val = make_df_parallel(setting='psd_dB', names=X_val)
df_test = make_df_parallel(setting='raw', names=X_test)

In [23]:
y_train = y_train + y_val

## MODELING

## k-Nearest Neighbors

In [24]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()#with_std=True)

x_train = sc.fit_transform(df_train.values)
x_test = sc.fit_transform(df_test.values)


In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

# grid_params = {
#     'n_neighbors': [11, 13, 15],
#     'weights': ['uniform', 'distance'],
#     'metric': ['euclidean', 'manhattan']
# }

# gs = GridSearchCV(KNeighborsClassifier(),
#                  grid_params,
#                  verbose=1,
#                  cv=5,
#                  n_jobs=-1)

# gs_results = gs.fit(x_train, y_train)

In [26]:
# plt.plot(x_train[0])

In [27]:
import sklearn
sklearn.__version__

'0.21.3'

In [ ]:
# from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

# classifier = LogisticRegression( solver='saga', penalty='elasticnet', l1_ratio=0.3, multi_class='ovr', max_iter=300, n_jobs=-1)
# from sklearn.neural_network import MLPClassifier
# classifier = MLPClassifier(solver='lbfgs', alpha=1e-5, max_iter=1000, activation='relu',
#                             hidden_layer_sizes=(30,), learning_rate='adaptive', verbose=True)

classifier = RandomForestClassifier(n_estimators=100, n_jobs=-1, )

classifier.fit(x_train, y_train)

In [ ]:
# classifier = gs_results.best_estimator_

# classifier.fit(df_train.values, y_train)
y_pred = classifier.predict(df_test.values)

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,balanced_accuracy_score
import seaborn as sns
sns.set(font_scale=2.) 

# Calculating confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Creating heatmap
plt.figure(figsize=(24,18))
cmap = sns.cubehelix_palette(50, hue=0.05, rot=0, light=0.9, dark=0, as_cmap=True)
sns.heatmap(cm, annot=True, fmt='g', cmap=cmap)

# Saving figure
# plt.savefig('/home/kalfasyan/Desktop/confusion_mat_knn.eps', format='eps')
ac = balanced_accuracy_score(y_test, y_pred)
print('Balanced Accuracy Score: {}'.format(ac))